Erstelle eine neue Conda Umgebung wanderwege

In [ ]:
conda create --name wanderwege
conda activate wanderwege

Wir benutzen Python 3.12.2 mit folenden Paketen

In [ ]:
conda install pandas
conda install -c anaconda sqlalchemy
conda install -c conda-forge overpy
conda install -c anaconda mysql-connector-python
conda install -c conda-forge pymysql
pip install overpy

Wir importieren die Daten von Overpass als json und überführen sie in ein Dataframe

In [3]:
import overpy
import pandas as pd

# Initialisiere die Overpass API mit einer benutzerdefinierten URL
api = overpy.Overpass(url="http://overpass.osm.ch/api/interpreter")

# Overpass Query für Wanderwege innerhalb der Schweiz. Mit Center erhalten wir die Koordinaten in der Mitte eines Wnaderweges
query = """
[out:json][timeout:25];
relation["route"="hiking"]
["name"]
["name"!~"fixme", i]
["network"="lwn"]
["osmc:symbol"~"yellow::yellow_diamond|red:white:red_bar|yellow:white:yellow_diamond|blue:white:blue_bar"]
(45.8899, 6.0872, 47.8085, 10.4921);
out center tags;
"""

# Führe die Anfrage aus
result = api.query(query)

# Liste zum Speichern der extrahierten Informationen für das DataFrame
data = []

# Iteration über alle relations
for relation in result.relations:
    
    # Center ist ein Tupel mit langitute und longitude, wir möchten nur einen wert
    lat = getattr(relation, 'center_lat')
    lon = getattr(relation, 'center_lon')

    # Füge die Relation-Daten zu einer Liste hinzu,
    relation_data = {
    'id': relation.id,
    'name': relation.tags.get('name'),
    'lat': lat,
    'lon': lon
    }

    # Jedes Dictionary wird in die Liste 'data' eingefügt, sodass die Daten gesammelt werden
    data.append(relation_data)

# Wenn alle Relationen verarbeitet wurden, erstelle das DataFrame
df = pd.DataFrame(data)

# Zeige das DataFrame an
print(df)


            id                                          name         lat  \
0        22614  Nationalpark Wanderroute 15 (Munt la Schera)  46.6501430   
1       103607                                 Wanderwege SG  47.4309774   
2       118485                                  Rund um Sulz  47.5352432   
3       120120                          Les Praises-La Casba  46.8326800   
4       120121                          La Casba-Les Avattes  46.8356755   
...        ...                                           ...         ...   
7697  17940255                      Urzlenboden - Stutzhütte  47.1000278   
7698  17942284                               Val d Osola rot  46.3366257   
7699  17948858             blaues Minus Kressbronn - Hörbolz  47.5977302   
7700  17989991                                 Vna - Griosch  46.8617089   
7701  17992364                                      Paneneve  46.2746355   

             lon  
0     10.2301992  
1      9.6201700  
2      8.0967558  
3      6.51

Die Resultate sind nun in einem Dataframe und können in die mysql Datenbank gespeichert werden, 
dazu erstelle ich eine mysql Datenbank

In [ ]:
import mysql.connector

mydb = mysql.connector.connect(
  host="127.0.0.1",
  user="root",
  passwd=""
)

print(mydb)

Nun lade ich die Daten in die Datenbank


In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# Verbindung zur MySQL-Datenbank herstellen (ersetze Platzhalter mit deinen Daten)
user = 'root'        # MySQL-Benutzername
password = 'mysqlfortuna'  # MySQL-Passwort
host = '127.0.0.1'   # Server (bei lokalen Installationen ist das oft 'localhost')
database = 'mydatabase'  # Name der Datenbank, in die du schreiben möchtest

# Erstelle die Verbindungs-Engine
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{database}")

# DataFrame in die MySQL-Datenbank-Tabelle schreiben (wenn die Tabelle nicht existiert, wird sie erstellt)
df.to_sql(name='Wanderwege', con=engine, if_exists='replace', index=False)

print("DataFrame erfolgreich in die MySQL-Datenbank geladen!")